### Load datasets into BQ

In [1]:
dataset_id = "JHU_Daily_Reports_staging"

In [2]:
dataset_id2 = "US_Census_Bureau_staging"

In [3]:
!bq --location=US mk --dataset {dataset_id}

BigQuery error in mk operation: Dataset 'lucid-
hook-288402:JHU_Daily_Reports_staging' already exists.


In [4]:
!bq --location=US mk --dataset {dataset_id2}

BigQuery error in mk operation: Dataset 'lucid-
hook-288402:US_Census_Bureau_staging' already exists.


In [5]:
!bq --location=US load --autodetect --skip_leading_rows=1 \
--source_format=CSV {dataset_id}.COVID19_USdeaths 'gs://utbeemers_covid19/covid19/time_series_covid_19_deaths_US.csv'

Waiting on bqjob_r17cdb34bf54c8368_000001759ab8db42_1 ... (4s) Current status: DONE   


In [6]:
!bq --location=US load --autodetect --skip_leading_rows=1 \
--source_format=CSV {dataset_id2}.Peopleover65_USdeaths 'gs://utbeemers_covid19/secondary dataset/GCT0103.csv'

Waiting on bqjob_r1a3ccc6105d3fcbf_000001759ab8f70b_1 ... (1s) Current status: DONE   


In [7]:
!bq --location=US load --autodetect --skip_leading_rows=1 \
--source_format=CSV {dataset_id2}.Peopleover85_USdeaths 'gs://utbeemers_covid19/secondary dataset/GCT0104.csv'

Waiting on bqjob_r461461ecac03343_000001759ab9047f_1 ... (1s) Current status: DONE   


### Perform 10 queries

In [23]:
#Showing five districts of Texas from JHU dataset and ordering by population, does not account for COVID19 deaths

In [24]:
%%bigquery
select * from JHU_Daily_Reports_staging.COVID19_USdeaths where Province_State = 'Texas' order by Population limit 5

UID iso2 iso3  code3     FIPS      Admin2 Province_State  \
0  84090048   US  USA    840  90048.0  Unassigned          Texas   
1  84090048   US  USA    840  90048.0  Unassigned          Texas   
2  84080048   US  USA    840  80048.0   Out of TX          Texas   
3  84090048   US  USA    840  90048.0  Unassigned          Texas   
4  84080048   US  USA    840  80048.0   Out of TX          Texas   

  Country_Region  Lat  Long_  ... _10_25_20  _10_26_20  _10_27_20  _10_28_20  \
0             US  0.0    0.0  ...         0          0          0          0   
1             US  0.0    0.0  ...         0          0          0          0   
2             US  0.0    0.0  ...         0          0          0          0   
3             US  0.0    0.0  ...         0          0          0          0   
4             US  0.0    0.0  ...         0          0          0          0   

   _10_29_20  _10_30_20  _10_31_20  _11_1_20  _11_2_20  _11_3_20  
0          0          0          0         0         0         0  
1          0          0          0         0         0         0  
2          0          0          0         0         0         0  
3          0          0          0         0         0         0  
4          0          0          0         0         0         0  

[5 rows x 299 columns]

In [9]:
# Showing each of COVID19 deaths across 10 of the district places in Texas from JHU dataset on 8/3/20

In [10]:
%%bigquery
select count(_8_3_20) as count from JHU_Daily_Reports_staging.COVID19_USdeaths where Province_State = 'Texas' group by Admin2 limit 10

count
0      3
1      3
2      3
3      3
4      3
5      3
6      3
7      3
8      3
9      3

In [11]:
# Showing total COVID19 deaths across the districts of Texas from JHU dataset on 8/3/20

In [12]:
%%bigquery
select sum(_8_3_20) as sum from JHU_Daily_Reports_staging.COVID19_USdeaths where Province_State = 'Texas' group by Admin2 order by Admin2

sum
0     27
1     18
2    108
3     36
4      0
..   ...
251   15
252    3
253   12
254    6
255   15

[256 rows x 1 columns]

In [27]:
#showing first ten results of population greater than >40000 and returning max of COVID19 deaths from JHU dataset inthe US on 8/3/20

In [25]:
%%bigquery
select max(_8_3_20) as max from JHU_Daily_Reports_staging.COVID19_USdeaths where Country_Region = 'US' group by Population having (Population > 40000) limit 10

max
0   20
1    6
2    5
3    2
4    8
5   16
6   49
7    8
8   11
9    4

In [31]:
#Showing five results of Alabama district with increasing latitude from JHU Datset, not accoutning for COVID19 deaths

In [32]:
%%bigquery
select * from JHU_Daily_Reports_staging.COVID19_USdeaths where Province_State = 'Alabama' order by Lat limit 5

UID iso2 iso3  code3     FIPS      Admin2 Province_State  \
0  84080001   US  USA    840  80001.0   Out of AL        Alabama   
1  84080001   US  USA    840  80001.0   Out of AL        Alabama   
2  84090001   US  USA    840  90001.0  Unassigned        Alabama   
3  84080001   US  USA    840  80001.0   Out of AL        Alabama   
4  84090001   US  USA    840  90001.0  Unassigned        Alabama   

  Country_Region  Lat  Long_  ... _10_25_20  _10_26_20  _10_27_20  _10_28_20  \
0             US  0.0    0.0  ...         0          0          0          0   
1             US  0.0    0.0  ...         0          0          0          0   
2             US  0.0    0.0  ...         0          0          0          0   
3             US  0.0    0.0  ...         0          0          0          0   
4             US  0.0    0.0  ...         0          0          0          0   

   _10_29_20  _10_30_20  _10_31_20  _11_1_20  _11_2_20  _11_3_20  
0          0          0          0         0         0         0  
1          0          0          0         0         0         0  
2          0          0          0         0         0         0  
3          0          0          0         0         0         0  
4          0          0          0         0         0         0  

[5 rows x 299 columns]

In [13]:
# areas with highest percentage of people over 65

In [14]:
%%bigquery
select * from US_Census_Bureau_staging.Peopleover65_USdeaths
where ESTIMATE > 15
order by ESTIMATE desc
limit 10

TBLID           GEOID                                            GEONAME  \
0  R0103    40000US45937  Lady Lake--The Villages, FL Urbanized Area (2010)   
1  R0103    40000US45937  Lady Lake--The Villages, FL Urbanized Area (2010)   
2  R0103    40000US45937  Lady Lake--The Villages, FL Urbanized Area (2010)   
3  R0103  97000US1201800             Sumter County School District, Florida   
4  R0103    31000US45540                        The Villages, FL Metro Area   
5  R0103  97000US1201800             Sumter County School District, Florida   
6  R0103    05000US12119                             Sumter County, Florida   
7  R0103    05000US12119                             Sumter County, Florida   
8  R0103    31000US45540                        The Villages, FL Metro Area   
9  R0103    05000US12119                             Sumter County, Florida   

   PROFLN  ESTIMATE MG_ERROR  
0       3      63.7   +/-5.2  
1       3      63.7   +/-5.2  
2       3      63.7   +/-5.2  
3       3      59.5   +/-0.9  
4       3      59.5   +/-0.9  
5       3      59.5   +/-0.9  
6       3      59.5   +/-0.9  
7       3      59.5   +/-0.9  
8       3      59.5   +/-0.9  
9       3      59.5   +/-0.9

In [15]:
# areas with lowest percentages of people over 65

In [16]:
%%bigquery
select * from US_Census_Bureau_staging.Peopleover65_USdeaths
where ESTIMATE < 15
order by ESTIMATE
limit 10

TBLID           GEOID                                            GEONAME  \
0  R0103  79500US4804626  Harris County (North)--Houston City (North)--I...   
1  R0103  79500US4804614  Houston City (West)--Westpark Tollway, Between...   
2  R0103  79500US4804614  Houston City (West)--Westpark Tollway, Between...   
3  R0103  79500US4804626  Harris County (North)--Houston City (North)--I...   
4  R0103  79500US4804626  Harris County (North)--Houston City (North)--I...   
5  R0103  79500US4804614  Houston City (West)--Westpark Tollway, Between...   
6  R0103  97000US4840950       Southwest Independent School District, Texas   
7  R0103  16000US4944320                                    Lehi city, Utah   
8  R0103  16000US4944320                                    Lehi city, Utah   
9  R0103  97000US4840950       Southwest Independent School District, Texas   

   PROFLN  ESTIMATE MG_ERROR  
0       3       5.5   +/-1.3  
1       3       5.5   +/-1.1  
2       3       5.5   +/-1.1  
3       3       5.5   +/-1.3  
4       3       5.5   +/-1.3  
5       3       5.5   +/-1.1  
6       3       5.9   +/-1.8  
7       3       5.9   +/-1.5  
8       3       5.9   +/-1.5  
9       3       5.9   +/-1.8

In [17]:
# Grouping by geography name, returns a list of Geonames with a given
# margin of error in the data estimates

In [18]:
%%bigquery
select GEONAME from US_Census_Bureau_staging.Peopleover65_USdeaths
where MG_ERROR != '*****' or null
group by GEONAME
limit 10

GEONAME
0                         Gwinnett County, Georgia
1         Gwinnett County School District, Georgia
2   Prince William County Public Schools, Virginia
3                                    Utah -- Urban
4         Utah -- In metropolitan statistical area
5                Houston, TX Urbanized Area (2010)
6                        Tulare County, California
7               Mecklenburg County, North Carolina
8       Dallas-Fort Worth-Arlington, TX Metro Area
9  Houston-The Woodlands-Sugar Land, TX Metro Area

In [19]:
# areas with highest percentage of people over 85

In [20]:
%%bigquery
select * from US_Census_Bureau_staging.Peopleover85_USdeaths
where ESTIMATE > 5
order by ESTIMATE desc
limit 10

TBLID           GEOID                                            GEONAME  \
0  R0104  79500US1209907  Palm Beach County (Southeast)--Greater Delray ...   
1  R0104  79500US1209907  Palm Beach County (Southeast)--Greater Delray ...   
2  R0104  79500US1209907  Palm Beach County (Southeast)--Greater Delray ...   
3  R0104  79500US1211502  Sarasota County (Central & Gulf Coast) PUMA, F...   
4  R0104  79500US1211502  Sarasota County (Central & Gulf Coast) PUMA, F...   
5  R0104  79500US1211502  Sarasota County (Central & Gulf Coast) PUMA, F...   
6  R0104  79500US0400130  Maricopa County--El Mirage City & Sun City PUM...   
7  R0104  79500US0400130  Maricopa County--El Mirage City & Sun City PUM...   
8  R0104  79500US0400130  Maricopa County--El Mirage City & Sun City PUM...   
9  R0104  79500US1207101  Lee County (South & West)--Barrier Islands Reg...   

   PROFLN  ESTIMATE MG_ERROR  
0       4       7.8   +/-1.5  
1       4       7.8   +/-1.5  
2       4       7.8   +/-1.5  
3       4       7.6   +/-1.4  
4       4       7.6   +/-1.4  
5       4       7.6   +/-1.4  
6       4       7.5   +/-1.1  
7       4       7.5   +/-1.1  
8       4       7.5   +/-1.1  
9       4       6.9   +/-1.4

In [21]:
# areas with lowest percentages of people over 85

In [22]:
%%bigquery
select * from US_Census_Bureau_staging.Peopleover85_USdeaths
where ESTIMATE < 5
order by ESTIMATE
limit 10

TBLID           GEOID                                            GEONAME  \
0  R0104  79500US4804626  Harris County (North)--Houston City (North)--I...   
1  R0104  79500US4804626  Harris County (North)--Houston City (North)--I...   
2  R0104  79500US4804626  Harris County (North)--Houston City (North)--I...   
3  R0104  97000US4816620       Del Valle Independent School District, Texas   
4  R0104  79500US0400121  Phoenix (Southwest) & Tolleson Cities PUMA, Ar...   
5  R0104  97000US4816620       Del Valle Independent School District, Texas   
6  R0104  79500US4804634  Harris County (West)--Houston City (West)--Nor...   
7  R0104  79500US4804634  Harris County (West)--Houston City (West)--Nor...   
8  R0104  79500US0400121  Phoenix (Southwest) & Tolleson Cities PUMA, Ar...   
9  R0104  79500US4804634  Harris County (West)--Houston City (West)--Nor...   

   PROFLN  ESTIMATE MG_ERROR  
0       4       0.0   +/-0.1  
1       4       0.0   +/-0.1  
2       4       0.0   +/-0.1  
3       4       0.1   +/-0.1  
4       4       0.1   +/-0.1  
5       4       0.1   +/-0.1  
6       4       0.1   +/-0.1  
7       4       0.1   +/-0.1  
8       4       0.1   +/-0.1  
9       4       0.1   +/-0.1